# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pickle
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sqlite3
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.metrics import classification_report, accuracy_score
import sys
import time

/Users/nadimkawwa/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nadimkawwa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql('messages', con= engine)


#alternatively use a query
# # connect to the database
# conn = sqlite3.connect('DisasterResponse')

# #read the table
# df= pd.read_sql_query('SELECT * FROM messages', conn)

df.shape

(26180, 40)

In [3]:
#select messages as feature space
X= df['message']
#select targets
Y= df.iloc[:, 4:]

In [4]:
#make function to load data

def load_data(database_filepath='messages.db'):
    """
    Input:
    filepath of a sql database
    """
    
    #create engine
    engine = create_engine('sqlite:///messages.db')
    
    #read sql
    df = pd.read_sql('messages', con= engine)
    
    #select messages as feature space
    X= df['message']
    #select targets
    Y= df.iloc[:, 4:]
    #get target categories
    categories = Y.columns.tolist()
    
    return X, Y, categories

In [5]:
#test run
X, Y, categories= load_data()

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    #detect urls
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    #detect all urls
    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        #replace all url with a placeholder
        text= text.replace(url, 'urlplaceholder')
        
    #normalize the text
    text = re.sub(r"[^a-zA-Z0-0]", ' ', text.lower())
        
    #tokenize text and assume english
    tokens = word_tokenize(text, language='english')
    
    # Remove Stopwords
    tokens = [x for x in tokens if x not in stopwords.words('english')]
    
    #remove words shorter than 2 characers
    tokens = [x for x in tokens if len(x)>2]
    
    #instantaite lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #use list comprehemsion to save on memory
    lemmed = [lemmatizer.lemmatize(tok, pos = 'n').strip() for tok in tokens]
    lemmed = [lemmatizer.lemmatize(tok, pos = 'v').strip() for tok in lemmed]
    
    return lemmed


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#instantiate a pipeline
model = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))

])

In [8]:
#evaluate keys to be used in make_pipe function
model.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
#split data in train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [10]:
start = time.time()
model.fit(X_train, Y_train)
delta = (time.time() - start)/60.0
print("Model trained in {:.2f} minutes".format(delta))

Model trained in 2.55 minutes


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
Y_pred = model.predict(X_test)
# Calculate the accuracy for each 
for i in range(len(categories)):
    cat = categories[i]
    report = classification_report(Y_test.iloc[:, i].values, Y_pred[:,i])
    score = accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])
    print("Category: \t {}".format(cat))
    print(report)
    print("Accuracy: \t {:.2f}\n".format(score))

Category: 	 related
             precision    recall  f1-score   support

          0       0.61      0.52      0.56      1178
          1       0.86      0.90      0.88      4010
          2       0.34      0.50      0.40        48

avg / total       0.80      0.81      0.80      5236

Accuracy: 	 0.81

Category: 	 request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      4356
          1       0.82      0.45      0.58       880

avg / total       0.89      0.89      0.88      5236

Accuracy: 	 0.89

Category: 	 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5210
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5236

Accuracy: 	 1.00

Category: 	 aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3071
          1       0.75      0.59      0.66      2165

avg /

/Users/nadimkawwa/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
def make_model():
    
    """
    Inputs:
    - None
    Output:
    - Multioutput RF classifier gridsearch object
    
    """
    
    #instantiate a pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    
    ])

    #set up paramaters for gridh search
    #uncomment cell blocks below if not in a rush

    parameters = {
        'vect__ngram_range': [(1,1), (1,2)],
        #'vect__max_df': [0.5, 0.75, 1.0],
        #'vect__max_features': [None, 20, 50],
        'tfidf__use_idf': [True, False],
        'clf__estimator__min_samples_split': [2, 3, 4]
                 }
    cv = GridSearchCV(pipeline, parameters)
    
    return cv

In [13]:
#create intance of model
model = make_model()

#record start
start = time.time()
#fit to training data
model.fit(X_train, Y_train)

delta = (time.time() - start)/60.0
print("Model trained in {:.2f} minutes".format(delta))

Model trained in 132.56 minutes


In [14]:
#get best parameters
model.best_params_

{'clf__estimator__min_samples_split': 2,
 'tfidf__use_idf': False,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
def model_evaluate(model, X_test, Y_test, categories):
    """
    Prints out precision, recall, and f1-score for trained model
    Inputs:
    - model: a trained model
    - X_test: the test features
    - Y_test: the test targets
    - categories: list of categories
    """
    
    Y_pred = model.predict(X_test)
    # Calculate the accuracy for each category
    for i in range(len(categories)):
        cat = categories[i]
        report = classification_report(Y_test.iloc[:, i].values, Y_pred[:,i])
        score = accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])
        print("Category: \t {}".format(cat))
        print(report)
        print("Accuracy: \t {:.2f}\n".format(score))

In [18]:
model_evaluate(model, X_test, Y_test, categories)

Category: 	 related
             precision    recall  f1-score   support

          0       0.63      0.52      0.57      1178
          1       0.86      0.91      0.89      4010
          2       0.35      0.52      0.42        48

avg / total       0.81      0.82      0.81      5236

Accuracy: 	 0.82

Category: 	 request
             precision    recall  f1-score   support

          0       0.89      0.98      0.94      4356
          1       0.85      0.43      0.57       880

avg / total       0.89      0.89      0.88      5236

Accuracy: 	 0.89

Category: 	 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5210
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5236

Accuracy: 	 1.00

Category: 	 aid_related
             precision    recall  f1-score   support

          0       0.74      0.88      0.81      3071
          1       0.77      0.57      0.66      2165

avg /

/Users/nadimkawwa/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [25]:
def save_model(model, filepath):
    '''
    Dump model as pickel
    Input: 
    - model: Model to be saved
    - filepath: path to model 
    '''
    pickle.dump(model, open(filepath, "wb"))
    print("model saved as {}".format(filepath))

In [26]:
save_model(model, 'saved_model.pickle')

model saved as saved_model.pickle


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [23]:
# load it again make sure it works
with open('saved_model.pickle', 'rb') as file:
    model_loaded = pickle.load(file)

In [24]:
model_evaluate(model_loaded, X_test, Y_test, categories)

Category: 	 related
             precision    recall  f1-score   support

          0       0.63      0.52      0.57      1178
          1       0.86      0.91      0.89      4010
          2       0.35      0.52      0.42        48

avg / total       0.81      0.82      0.81      5236

Accuracy: 	 0.82

Category: 	 request
             precision    recall  f1-score   support

          0       0.89      0.98      0.94      4356
          1       0.85      0.43      0.57       880

avg / total       0.89      0.89      0.88      5236

Accuracy: 	 0.89

Category: 	 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5210
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5236

Accuracy: 	 1.00

Category: 	 aid_related
             precision    recall  f1-score   support

          0       0.74      0.88      0.81      3071
          1       0.77      0.57      0.66      2165

avg /

/Users/nadimkawwa/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Category: 	 death
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      5003
          1       0.76      0.13      0.23       233

avg / total       0.95      0.96      0.95      5236

Accuracy: 	 0.96

Category: 	 other_aid
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      4562
          1       0.57      0.05      0.10       674

avg / total       0.84      0.87      0.82      5236

Accuracy: 	 0.87

Category: 	 infrastructure_related
             precision    recall  f1-score   support

          0       0.94      1.00      0.97      4942
          1       0.25      0.00      0.01       294

avg / total       0.91      0.94      0.92      5236

Accuracy: 	 0.94

Category: 	 transport
             precision    recall  f1-score   support

          0       0.95      1.00      0.98      4988
          1       0.71      0.05      0.09       248

avg / total       0.94      0.95      0.93 